In [ ]:
import sales_prediction.step.pred_tabular as pt
from sales_prediction.sales_prediction import EarlyStopping
from sales_prediction.preproc_tabular import get_tabular
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
desc_path= 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni.pt'
desc_tot_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni_tot.pt'

data_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_week.csv'
data_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_month.csv'
data_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_season.csv'

noneg_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_week.csv'
noneg_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_month.csv'
noneg_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_season.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

modello = pt.create_model(0, 0).to(device)

batch_size = 128
epochs = 40
criterion = nn.MSELoss()
optimizer = AdamW(modello.parameters(), lr=0.01) 
early_stop = EarlyStopping(5, 0.001) 

data, _ , _, target = get_tabular(noneg_week, desc_tot_path)
train, val = pt.getDataset(data, target, batch_size, 1)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    pt.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2, bias = pt.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)

    if early_stp:
        print('Early Stop attivato.')
        break
    
print("Fatto!")